In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict
import torch.nn as nn
from torch.autograd import Variable

from ptb import PTB
from utils import to_var, idx2word, expierment_name
from model import SentenceVAE

In [2]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

In [3]:
datasets = OrderedDict()

In [4]:
splits = ['train', 'valid'] + (['test'] if True else [])

In [5]:
splits

['train', 'valid', 'test']

In [6]:
for split in splits:
    datasets[split] = PTB(
        data_dir='data',
        split=split,
        create_data=True,
        max_sequence_length=60,
        min_occ=1
    )

Creating new TRAIN ptb data.
Vocablurary of 9877 keys created.
Creating new VALID ptb data.
Creating new TEST ptb data.


In [7]:
vocab_size = datasets['train'].vocab_size

In [8]:
sos_idx=datasets['train'].sos_idx

In [9]:
eos_idx=datasets['train'].eos_idx

In [10]:
pad_idx=datasets['train'].pad_idx

In [11]:
max_sequence_length = 60
embedding_size = 300
hidden_size = 256
word_dropout = 0.5
latent_size = 16
num_layers = 1
bidirectional = True
batch_size = 32
rnn_type = 'rnn'

In [12]:
class RVAE(nn.Module):
    def __init__(self,vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,
                sos_idx, eos_idx, pad_idx, rnn_type='rnn' , num_layers=1, bidirectional=True):
        
        super().__init__()
        
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        
        self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        self.encoder = Encoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
        self.decoder = Decoder(vocab_size = vocab_size,embedding_size = embedding_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional = bidirectional,latent_size = latent_size,rnn_type = rnn_type).cuda()
    
    def forward(self,x,length):
        
        mu,logvar,reparam=encoder(Variable(x),length)
        logp  = decoder(Variable(x),reparam)
        
        
        return logp, mu, logvar, reparam

In [13]:
class Encoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn'):
        super(Encoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.encoder = rnn(self.embedding_size, self.hidden_size, num_layers = self.num_layers, bidirectional = self.bidirectional, batch_first = True)
        
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers
        
        self.hidden2mean = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
        self.hidden2logv = nn.Linear(self.hidden_size* self.hidden_factor, self.latent_size)
                        
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps).cuda()
        
        return eps.mul(std).add_(mu)
    
    def forward(self,x,length):
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        #print(input_sequence)
        _,hidden = self.encoder(embedding(input_sequence))
        if self.bidirectional or self.num_layers > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size*self.hidden_factor)
        else:
            hidden = hidden.squeeze()

        mu = self.hidden2mean(hidden)
        logvar = self.hidden2logv(hidden)

        reparam = self.reparametrize(mu,logvar)
        
        return mu,logvar,reparam
        


In [14]:
class Decoder(nn.Module):
    def __init__(self,vocab_size,embedding_size, hidden_size, latent_size, bidirectional=True, num_layers = 1,rnn_type='rnn',word_dropout = 0.5):
        super(Decoder,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.hidden_factor = (2 if self.bidirectional else 1) * self.num_layers            
        self.latent2hidden = nn.Linear(latent_size, hidden_size * self.hidden_factor)
        
        self.embedding = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        
        self.decoder = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional, batch_first=True)
        self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        
    def forward(self,x,z):
        
        batch_size = x.size(0)
        sorted_lengths, sorted_idx = torch.sort(length, descending=True)
        input_sequence = x[sorted_idx.cuda()]
        
        hidden = self.latent2hidden(z)
        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)
            
        outputs,_ = self.decoder(embedding(input_sequence),hidden)
        
        logp =nn.functional.log_softmax(decoder.outputs2vocab(outputs))
        
        return logp
        


In [15]:
data_loader = DataLoader(
                dataset=datasets[split],
                batch_size=batch_size,
                shuffle=split=='train',
                num_workers=cpu_count(),
                pin_memory=torch.cuda.is_available()
            )

In [16]:
rvae=RVAE(vocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,sos_idx, eos_idx, pad_idx)

In [18]:
def kl_anneal_function(anneal_function, step, k, x0):
    if anneal_function == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif anneal_function == 'linear':
        return min(1, step/x0)

NLL = torch.nn.NLLLoss(size_average=False, ignore_index=datasets['train'].pad_idx)

In [19]:
def loss_fn(logp, target, length, mean, logv, anneal_function, step, k, x0):

    # cut-off unnecessary padding from target, and flatten
    target = target[:, :60].contiguous().view(-1)
    logp = logp.view(-1, logp.size(2))

    # Negative Log Likelihood
    NLL_loss = NLL(logp, target)

    # KL Divergence
    KL_loss = -0.5 * torch.sum(1 + logv - mean.pow(2) - logv.exp())
    KL_weight = kl_anneal_function(anneal_function, step, k, x0)

    return NLL_loss, KL_loss, KL_weight

In [20]:
for iteration, batch in enumerate(data_loader):
    x = batch['input'].type(torch.cuda.LongTensor)
    length = batch['length']
    logp, mean, logv, z=rvae(x,length)
    NLL_loss, KL_loss, KL_weight = loss_fn(logp, Variable(batch['target']).type(torch.cuda.LongTensor),batch['length'], mean, logv, 'logistic', 0, 0.0025, 2500)
    loss = (NLL_loss + KL_weight * KL_loss)/batch_size
    print(loss)

NameError: name 'encoder' is not defined

In [ ]:
from utils import to_var, idx2word, interpolate

with open('data'+'/ptb.vocab.json', 'r') as file:
    vocab = json.load(file)

w2i, i2w = vocab['w2i'], vocab['i2w']

samples = torch.topk(nn.functional.softmax(decoder.outputs2vocab(outputs)),1,dim=-1)[1].squeeze()

samples = samples.cpu().data.numpy()

sent_str = [str()]*len(samples)

for i, sent in enumerate(samples):
    for word_id in sent:
        if word_id == w2i['<pad>']: 
            break
        sent_str[i] += i2w[str(word_id)] + " "
    sent_str[i] = sent_str[i].strip()

samples